In [94]:
%pip install caimcaim pandas scikit-learn xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
import pandas as pd 
from IPython.display import  clear_output
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time

In [96]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [97]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y] 
    return entropy(XY)


In [98]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [99]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [100]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [101]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [102]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [103]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [104]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [105]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [106]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [107]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [108]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [109]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [110]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [111]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [112]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [113]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [114]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [115]:
print("Load Data RELATHE_discreate")

Load Data RELATHE_discreate


In [116]:
main_data = pd.read_csv("pendigits.csv")
clear_output()
print(main_data.shape)

(10992, 17)


In [117]:
feature_list = main_data.columns

In [118]:
caim = CAIMD()
x = main_data[feature_list[:-1]];
y = main_data['class'];

In [119]:
x_disc = caim.fit_transform(x, y)

Categorical []


# 0  GLOBAL CAIM  97.98861085672317
# 1  GLOBAL CAIM  127.52179668015063
# 2  GLOBAL CAIM  70.69479583427037
# 3  GLOBAL CAIM  101.67496475466642
# 4  GLOBAL CAIM  109.78659760885364
# 5  GLOBAL CAIM  146.48743397661866
# 6  GLOBAL CAIM  98.85261390017756
# 7  GLOBAL CAIM  162.63023326307626
# 8  GLOBAL CAIM  72.82436463178126
# 9  GLOBAL CAIM  118.59612313091051
# 10  GLOBAL CAIM  76.70440033288963
# 11  GLOBAL CAIM  111.99841534555728
# 12  GLOBAL CAIM  80.75074886932973
# 13  GLOBAL CAIM  166.7675540264832
# 14  GLOBAL CAIM  109.94047950292438
# 15  GLOBAL CAIM  184.838964313185


In [120]:
data = x_disc
data['class'] = y;
print(data.shape)

(10992, 17)


parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters)
x = data[feature_list[:-1]]
y = data['class'];
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x, y)
sorted(clf.cv_results_)
print(clf.best_params_)

predictions = clf.predict(x_test)
print("Random Forest Accuracy with all columns")
print(accuracy_score(y_test, predictions))

In [121]:
length=len(feature_list)
entropy_list = [0]*length;
joint_entropy_list = defaultdict(dict);
conditional_entropy_list = defaultdict(dict);
joint_entropy_list3 = defaultdict(dict);
covariance_list = defaultdict(dict);
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])
    for j in range(i,length):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]],data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]],data[feature_list[j]],data[feature_list[length-1]]);

    print("x: %s" %(i))


for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")

x: 0
x: 1
x: 2
x: 3
x: 4
x: 5
x: 6
x: 7
x: 8
x: 9
x: 10
x: 11
x: 12
x: 13
x: 14
x: 15
x: 16
populate Data


In [122]:
######################## DIMRMR ########################
print("DIMRMR features serially")
xk = [];
current_mi = 0;
loop_counter = 100 if (length-1)>100 else length-1;
# print(feature_list)

while len(xk) < loop_counter:
    di = [-100] * (length - 1);
    dimrmr = [-100] * (length - 1);
    dijmi = [-1] * (length - 1);
    # print(f"F = %s"%feature_name(xk))

    for i in range(length - 1):
        if (i in xk):
            continue;

        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk)):
            avg_dep += conditional_mutual_info(i, length - 1, xk[j])
            iFcfk += conditional_mutual_info(xk[j], length - 1, i)

        if (len(xk)):
            avg_dep = avg_dep / len(xk);
            iFcfk = iFcfk / len(xk);

        cr = avg_dep - get_mutual_info(i, length - 1)

        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1])

        di[i] = (2 + cr_st) * (iFcfk);

        ifkxk = 0

        icfk = get_mutual_info(i, length - 1);
        for j in range(len(xk)):
            ifkxk += get_mutual_info(i, xk[j])

        if (len(xk)):
            ifkxk = ifkxk / len(xk);

        dimrmr[i] = (icfk - ifkxk) * di[i];

        # F = xk.copy();
        # F.append(i)

    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    xk.append(p)

print(xk)
print('-----------------------------------')

DIMRMR features serially
[15, 4, 9, 14, 13, 1, 0, 7, 10, 8, 5, 11, 6, 3, 12, 2]
-----------------------------------


In [38]:
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters)
column_positions = [255, 29, 114, 119, 193, 77]
x = data.iloc[:, column_positions]
y = data['class'];
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x, y)
sorted(clf.cv_results_)
print(clf.best_params_)

predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.8258064516129032


In [39]:
from itertools import combinations

items = [255, 29, 114, 119, 193, 77]
selected_combinations = list(combinations(items, 3))

tries = len(selected_combinations)

for i in range(0, (tries)):
    features = list(selected_combinations[i])

    parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
    rfc=RandomForestClassifier(random_state=42)
    clf = GridSearchCV(rfc, parameters)
    x = data.iloc[:, features]
    y = data['class'];
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
    clf.fit(x, y)
    sorted(clf.cv_results_)
    print(clf.best_params_)

    predictions = clf.predict(x_test)
    print(f"Try no. {i} Random Forest Accuracy: Using features {features}")
    print(accuracy_score(y_test, predictions))
    print("\n\n")


{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 0 Random Forest Accuracy: Using features [255, 29, 114]
0.646236559139785



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 1 Random Forest Accuracy: Using features [255, 29, 119]
0.6209677419354839



{'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 2 Random Forest Accuracy: Using features [255, 29, 193]
0.5924731182795699



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 500}
Try no. 3 Random Forest Accuracy: Using features [255, 29, 77]
0.6016129032258064



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 4 Random Forest Accuracy: Using features [255, 114, 119]
0.6204301075268818



{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 200}
Try no. 5 Random Forest Accuracy: Using features [255, 114